# Goldstein-price function

**Contributed by**: Benoît Legat

In this example, we consider the minimization of the [Goldstein-price function](https://en.wikipedia.org/wiki/Test_functions_for_optimization).

In [1]:
using SumOfSquares
using DynamicPolynomials

Create *symbolic* variables (not JuMP *decision* variables)

In [2]:
@polyvar x[1:2]

(DynamicPolynomials.Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[x₁, x₂],)

To use Sum-of-Squares Programming, we first need to pick an SDP solver,
see [here](https://jump.dev/JuMP.jl/v1.12/installation/#Supported-solvers) for a list of the available choices.

In [3]:
import Clarabel
using Dualization
model = SOSModel(dual_optimizer(Clarabel.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Dual model with Clarabel attached

Create a JuMP decision variable for the lower bound

In [4]:
@variable(model, γ)

γ

`f(x)` is the Goldstein-Price function

In [5]:
f1 = x[1] + x[2] + 1
f2 = 19 - 14*x[1] + 3*x[1]^2 - 14*x[2] + 6*x[1]*x[2] + 3*x[2]^2
f3 = 2*x[1] - 3*x[2]
f4 = 18 - 32*x[1] + 12*x[1]^2 + 48*x[2] - 36*x[1]*x[2] + 27*x[2]^2
f = (1 + f1^2*f2) * (30 + f3^2*f4)

600 + 720x₂ + 720x₁ + 3060x₂² - 4680x₁x₂ + 1260x₁² + 12288x₂³ - 19296x₁x₂² + 7344x₁²x₂ - 1072x₁³ + 14346x₂⁴ - 23616x₁x₂³ + 7776x₁²x₂² + 5784x₁³x₂ - 2454x₁⁴ + 1944x₂⁵ - 11880x₁x₂⁴ + 5040x₁²x₂³ + 9840x₁³x₂² - 7680x₁⁴x₂ + 1344x₁⁵ - 4428x₂⁶ - 1188x₁x₂⁵ + 8730x₁²x₂⁴ + 1240x₁³x₂³ - 5370x₁⁴x₂² - 168x₁⁵x₂ + 952x₁⁶ - 648x₂⁷ + 1944x₁x₂⁶ + 3672x₁²x₂⁵ - 3480x₁³x₂⁴ - 4080x₁⁴x₂³ + 2592x₁⁵x₂² + 1344x₁⁶x₂ - 768x₁⁷ + 729x₂⁸ + 972x₁x₂⁷ - 1458x₁²x₂⁶ - 1836x₁³x₂⁵ + 1305x₁⁴x₂⁴ + 1224x₁⁵x₂³ - 648x₁⁶x₂² - 288x₁⁷x₂ + 144x₁⁸

Constraints `f(x) - γ` to be a sum of squares

In [6]:
con_ref = @constraint(model, f >= γ)
@objective(model, Max, γ)
optimize!(model)

-------------------------------------------------------------
           Clarabel.jl v0.6.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 45
  constraints   = 121
  nnz(P)        = 0
  nnz(A)        = 121
  cones (total) = 2
    : Zero        = 1,  numel = 1
    : PSDTriangle = 1,  numel = 120

settings:
  linear algebra: direct / qdldl, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-13, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       ga

The lower bound found is 3

In [7]:
solution_summary(model)

* Solver : Dual model with Clarabel attached

* Status
  Result count       : 1
  Termination status : ALMOST_OPTIMAL
  Message from the solver:
  "ALMOST_SOLVED"

* Candidate solution (result #1)
  Primal status      : NEARLY_FEASIBLE_POINT
  Dual status        : NEARLY_FEASIBLE_POINT
  Objective value    : 3.00006e+00
  Dual objective value : 3.00004e+00

* Work counters
  Solve time (sec)   : 1.90650e-02
  Barrier iterations : 16


The moment matrix is as follows, we can already see the global minimizer
`[0, -1]` from the entries `(2, 1)` and `(3, 1)`.
This heuristic way to obtain solutions to the polynomial optimization problem
is suggested in [Laurent2008; (6.15)](@cite).

In [8]:
ν = moment_matrix(con_ref)

MomentMatrix with row/column basis:
 MonomialBasis([1, x[2], x[1], x[2]^2, x[1]*x[2], x[1]^2, x[2]^3, x[1]*x[2]^2, x[1]^2*x[2], x[1]^3, x[2]^4, x[1]*x[2]^3, x[1]^2*x[2]^2, x[1]^3*x[2], x[1]^4])
And entries in a 15×15 SymMatrix{Float64}:
  1.0000000018198698     -0.9999904338316166     …   3.4104034285866996e-6
 -0.9999904338316166      0.9999813837943897         1.7043839977191587e-6
  1.401293336357993e-5   -1.3799592446004206e-5      7.451483754417287e-6
  0.9999813860035811     -0.9999722309458006         0.0002711976680818313
 -1.3823074641574693e-5   1.4109607254594264e-5      0.00028496583206889375
  8.140490999870537e-7    2.3020192377404057e-7  …   0.0005630993629247136
 -0.9999722273334571      0.9999633200853798        -0.00013925168205436442
  1.4102740426892929e-5  -1.3953042571734061e-5      2.2983476463909564e-6
  2.287994538559344e-7    3.914539624501102e-7       0.0005027103259083649
  1.14756778339497e-6     9.527302713889533e-7       0.0011791392891342552
  0.99996331

Many entries of the matrix actually have the same moment.
We can obtain the following list of these moments without duplicates
(ignoring when difference of entries representing the same moments is below `1e-5`)

In [9]:
μ = measure(ν, atol = 1e-5)

Measure{Float64, DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}, DynamicPolynomials.MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}}([1.0000000018198698, -0.9999904338316166, 1.401293336357993e-5, 0.9999813860035811, -1.3823074641574693e-5, 8.140490999870537e-7, -0.9999722273334571, 1.4102740426892929e-5, 2.287994538559344e-7, 1.14756778339497e-6  …  0.0011791392891342552, 1.4837695259617898, -0.38704799673282014, 0.5307619036335185, -0.3153797748347552, 0.6372092054366799, -0.1551786920868488, 0.8777398620413609, 0.2059975266673279, 1.422084768180202], DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[1, x₂, x₁, x₂², x₁x₂, x₁², x₂³, x₁x₂², x₁²x₂, x₁³  …  x₁⁷, x₂⁸, x₁x₂

The truncated moment matrix can then be obtained as follows

In [10]:
ν_truncated = moment_matrix(μ, monomials(x, 0:3))

MomentMatrix with row/column basis:
 MonomialBasis([1, x[2], x[1], x[2]^2, x[1]*x[2], x[1]^2, x[2]^3, x[1]*x[2]^2, x[1]^2*x[2], x[1]^3])
And entries in a 10×10 SymMatrix{Float64}:
  1.0000000018198698     -0.9999904338316166     …  1.14756778339497e-6
 -0.9999904338316166      0.9999813860035811        9.574747243383608e-7
  1.401293336357993e-5   -1.3823074641574693e-5     3.4104034285866996e-6
  0.9999813860035811     -0.9999722273334571        2.4810748752536637e-7
 -1.3823074641574693e-5   1.4102740426892929e-5     1.7043839977191587e-6
  8.140490999870537e-7    2.287994538559344e-7   …  7.451483754417287e-6
 -0.9999722273334571      0.9999633103973404        9.773522078850986e-5
  1.4102740426892929e-5  -1.3906964660954489e-5     0.0002711976680818313
  2.287994538559344e-7    3.931301909082682e-7      0.00028496583206889375
  1.14756778339497e-6     9.574747243383608e-7      0.0005630993629247136

Let's check if the flatness property is satisfied.
The rank of `ν_truncated` seems to be 1:

In [11]:
using LinearAlgebra
LinearAlgebra.svdvals(Matrix(ν_truncated.Q))
LinearAlgebra.rank(Matrix(ν_truncated.Q), rtol = 1e-3)
svdvals(Matrix(ν_truncated.Q))

10-element Vector{Float64}:
 3.9999129595584804
 0.0008735448885684927
 0.00017574267052825453
 7.615603475243653e-6
 6.052439999404675e-7
 3.320433002738225e-7
 1.1346221574951914e-7
 7.16763637145575e-8
 3.140108856202916e-8
 4.238028226828164e-9

The rank of `ν` is clearly higher than 1, closer to 3:

In [12]:
svdvals(Matrix(ν.Q))

15-element Vector{Float64}:
 5.17674761577772
 2.3189470566744617
 1.453558648851051
 0.0023462616989670462
 0.0006350998616532194
 0.00015172127913365465
 0.0001398235480826501
 9.002841822263696e-6
 6.652718664350959e-7
 2.7408971775687256e-7
 7.656812574907482e-8
 6.435800433409038e-8
 1.0665802363009165e-8
 6.706830024591651e-9
 3.6941439018433997e-10

Even if the flatness property is not satisfied, we can
still try extracting the minimizer with a low rank decomposition of rank 3.
We find the optimal solution again doing so:

In [13]:
atomic_measure(ν, FixedRank(3))

Atomic measure on the variables x[1], x[2] with 1 atoms:
 at [1.4273511730133842e-5, -0.9999906623763443] with weight 1.032287146482791

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*